Importamos dependencias

In [1]:
print("Hola")

Hola


In [ ]:
# # Warning control
# import warnings
# warnings.filterwarnings('ignore')

# Load environment variables
from dotenv import load_dotenv

load_dotenv()

import os
import yaml
from crewai import Agent, Task, Crew

Definimos el modelo

In [ ]:
from crewai import LLM

llm = LLM(
    model="openai/gpt-4o-mini", # call model by provider/model_name
    temperature=0.8,
    max_tokens=150,
    top_p=0.9,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stop=["END"],
    seed=42
)

Carga de instrucciones

In [ ]:
files = {
    'agents': 'config/hair_diagno_config/agents.yaml',
    'tasks': 'config/hair_diagno_config/tasks.yaml'
}

configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

agents_config = configs['agents']
tasks_config = configs['tasks']

Pydantic model for the diagnostic task

In [ ]:
from typing import List
from pydantic import BaseModel, Field, field_validator
from enum import Enum

class ToneTemperature(str, Enum):
    WARM = "cálido"
    COLD = "frío"
    NEUTRAL = "neutro"

class DamageLevel(str, Enum):
    LIGHT = "ligero"
    MODERATE = "moderado"
    SEVERE = "severo"

class Porosity(str, Enum):
    LOW = "baja"
    MEDIUM = "media"
    HIGH = "alta"

class HairCharacteristics(BaseModel):
    porosidad: Porosity = Field(..., description="Nivel de porosidad del cabello")
    grosor: str = Field(..., description="Grosor del cabello (fino, medio, grueso)")
    densidad: str = Field(..., description="Densidad del cabello (baja, media, alta)")
    textura: str = Field(..., description="Textura del cabello (liso, ondulado, rizado, etc.)")
    nivel_dano: DamageLevel = Field(..., description="Nivel de daño del cabello")

class HairDiagnostic(BaseModel):
    base_tone_height: int = Field(
        ..., 
        description="Altura del tono base en escala internacional (1-10)",
        ge=1, # greater than or equal
        le=10 # less than or equal
    )
    gray_hair_percentage: float = Field(
        ..., 
        description="Porcentaje aproximado de canas presentes",
        ge=0, 
        le=100
    )
    mid_ends_color_description: str = Field(
        ...,
        description="Descripción detallada del color en medios y puntas"
    )
    tone_temperature: ToneTemperature = Field(
        ...,
        description="Temperatura del tono actual (cálido, frío, neutro)"
    )
    tone_description: str = Field(
        ...,
        description="Descripción técnica completa del tono con matices específicos"
    )
    hair_characteristics: HairCharacteristics = Field(
        ...,
        description="Características estructurales del cabello"
    )
    recommendations: List[str] = Field(
        default=[],
        description="Recomendaciones basadas en el diagnóstico"
    )
 
    
    @field_validator('base_tone_height')
    def validate_tone_height(cls, v):
        if v < 1 or v > 10:
            raise ValueError('La altura del tono debe estar entre 1 y 10')
        return v
    
    class Config:
        schema_extra = {
            "example": {
                "base_tone_height": 6,
                "gray_hair_percentage": 50,
                "mid_ends_color_description": "Medios y puntas con decoloración previa a tono 8, presencia de reflejos amarillos intensos",
                "tone_temperature": "cálido",
                "tone_description": "Castaño medio con matices cobrizos y reflejos dorados",
                "hair_characteristics": {
                    "porosidad": "baja",
                    "grosor": "fino",
                    "densidad": "media",
                    "textura": "ondulado",
                    "nivel_dano": "moderado"
                },
                "recommendations": [
                    "Tratamiento de restructuración",
                    "Tinte demi-permanente para nivelar el color"
                ]
            }
        }

Agnents & Crewa